# Mean and Variance Estimates for Multiple Randomization Designs

In a two-sided marketplace, multiple sellers and multiple buyers interact. For instance, on an e-commerce platform,  which is a typical type of a two-sided marketplace, buyers see a sellers' offers and choose to buy or not. The platform wants to assess a new info policy's impact on a buyer's likelihood of buying an item. In a traditional A/B test, researchers can randomize buyers to A (treatment group) or B (control group) or sellers to A or B. These single randomization designs are traditional RCTs, and can lead to biased average treatment effect estimate. 

For example, the platform wants to recommend more TVs during black Friday to customers, and wants to test if the strategy is effective using A/B test. If researchers carry out an user side experiments, users in treatment and control group will compete for the same product. If treatment group users buy all TVs, which leads to control group users have nothing to buy, average treatment effect can be over-estimate. Therefore a two-sided A/B test is desired. 

We follow Bajari, Patrick, et al. (2021) (https://arxiv.org/pdf/2112.13495) formulas and provide treatment effect estimators for the following types.  


### Estimators

We have following types of estimators:
* $\tau(p^B,p^S)$: the treatment effect of treated buyer and treated seller pairs vs control buyer and control seller pair. The estimator is $\bar{\bar{Y_t}}-\bar{\bar{Y_{c}}}$, variance is 
\begin{equation}
\begin{aligned}
V(\tau(p^B, p^S)) = V(\bar{\bar{Y_t}}-\bar{\bar{Y_{c}}})=
&V(\bar{\bar{Y_t}})+V(\bar{\bar{Y_{c}}})-2C(\bar{\bar{Y_t}},\bar{\bar{Y_{c}}})\\
\end{aligned}
\end{equation}
* $\tau_{direct}$: the treatment effect of treated buyer and treated seller pairs vs control buyer and control seller pair, minus spill-over effect. The estimator is $\bar{\bar{Y_t}}-\bar{\bar{Y_{ib}}}-\bar{\bar{Y_{is}}}+\bar{\bar{Y_{c}}}$, variance is 
\begin{equation}
\begin{aligned}
V(\tau_{direct}) = V(\bar{\bar{Y_t}}-\bar{\bar{Y_{ib}}}-\bar{\bar{Y_{is}}}+\bar{\bar{Y_{c}}})=
&V(\bar{\bar{Y_t}})+V(\bar{\bar{Y_{ib}}})+V(\bar{\bar{Y_{is}}})+V(\bar{\bar{Y_{c}}})\\
&-2C(\bar{\bar{Y_t}},\bar{\bar{Y_{ib}}})-2C(\bar{\bar{Y_t}},\bar{\bar{Y_{is}}})+2C(\bar{\bar{Y_t}},\bar{\bar{Y_{c}}})\\
&+2C(\bar{\bar{Y_{ib}}},\bar{\bar{Y_{is}}})-2C(\bar{\bar{Y_{ib}}},\bar{\bar{Y_{c}}})\\
&-2C(\bar{\bar{Y_{is}}},\bar{\bar{Y_{c}}}))\\
\end{aligned}
\end{equation}
* $\tau^b_{spillover}$: spill-over effect from treated users to control users. The estimator is $\bar{\bar{Y_{ib}}}-\bar{\bar{Y_{c}}}$, variance is 
\begin{equation}
\begin{aligned}
V(\tau^b_{spillover}) = V(\bar{\bar{Y_{ib}}}-\bar{\bar{Y_{c}}})=
&V(\bar{\bar{Y_{ib}}})+V(\bar{\bar{Y_{c}}})-2C(\bar{\bar{Y_{ib}}},\bar{\bar{Y_{c}}})\\
\end{aligned}
\end{equation}
* $\tau^s_{spillover}$: spill-over effect from treated sellers to control sellers. The estimator is $\bar{\bar{Y_{is}}}-\bar{\bar{Y_{c}}}$, variance is 
\begin{equation}
\begin{aligned}
V(\tau^s_{spillover}) = V(\bar{\bar{Y_{is}}}-\bar{\bar{Y_{c}}})=
&V(\bar{\bar{Y_{is}}})+V(\bar{\bar{Y_{c}}})-2C(\bar{\bar{Y_{is}}},\bar{\bar{Y_{c}}})\\
\end{aligned}
\end{equation}

In formulas above, $V(\bar{\bar{Y_t}})$, $V(\bar{\bar{Y_{ib}}})$, $V(\bar{\bar{Y_{is}}})$, and $V(\bar{\bar{Y_c}})$ follows Lemma A.4. Covariance terms follows Lemma A.5.



In [1]:
# import sys
# print(sys.path)
# sys.path.append('/Users/bytedance/PycharmProjects/github/CausalMatch')

import causalmatch as causalmatch
from causalmatch import matching, gen_test_data, gen_test_data_mrd
from causalmatch import mrd

print('current version is: ',causalmatch.__version__)


import pandas as pd
import numpy as np
import statsmodels.api as sm


['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/bytedance/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages']
current version is:  0.0.5


In [2]:
# STEP 1: generate synthetic data of a two-sided experiment:
#         In the test dataset, shop_id mimics seller id in Bajari, Patrick, et al. (2021),
#         user_id mimics buyer id, treatment_u is the treatment status of the user_id,
#         treatment_s is the treatment status of the shop_id, y_overflow is the dependent variable
#         target to be estimated. 

df_raw = gen_test_data_mrd(n_shops = 1000
                           , n_users = 10
                           , ate   = 1 # ATE
                           , uflow = 0.5 # user side overflow effect
                           , sflow = 0.5 # seller side overflow effect
                           , fraction_bt = 0.5 # treatment group fraction
                           , fraction_st = 0.5 # control group fraction
                           , seed = 123456) # random seed number
df_raw.head()

,shop_id,user_id,treatment,treatment_u,treatment_s,error,status,y_clean,y_overflow
0,1,1,1,1,1,0.046911,t,1.046911,2.046911
1,2,1,0,1,0,-0.028286,ib,-0.028286,0.471714
2,3,1,1,1,1,-0.150906,t,0.849094,1.849094
3,4,1,1,1,1,-0.113563,t,0.886437,1.886437
4,5,1,0,1,0,0.121211,ib,0.121211,0.621211


In [3]:
# STEP 2: initialize a mrd object
mrd_obj = mrd(data = df_raw,
              idb = 'user_id',
              ids = 'shop_id',
              tb  = 'treatment_u',
              ts  = 'treatment_s',
              y   = 'y_overflow')

In [4]:
# STEP 3: calculate average treatment effect of four type
mrd_obj.ate2()

,parameters,mean,variance,t_stat,p_values
0,tau,2.000219,1.279201,1.768512,0.077052
1,tau_tdirect,0.999174,0.284924,1.871873,0.061297
2,tau_b_spillover,0.501487,0.079940,1.773693,0.076190
3,tau_s_spillover,0.499558,0.023354,3.268926,0.001089
